In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import os

In [4]:
# Ruta al dataset
DATASET_DIR = "../datasets"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)


Found 7475 images belonging to 3 classes.
Found 1868 images belonging to 3 classes.


In [6]:
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights="imagenet")
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])


In [7]:
from tensorflow.keras.callbacks import EarlyStopping

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)]
)

c:\Users\USUARIO\Desktop\Python-Flask\EcoEat\ml-model\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 244s 1s/step - accuracy: 0.7415 - loss: 0.5996 - val_accuracy: 0.9427 - val_loss: 0.1612
Epoch 2/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 153s 655ms/step - accuracy: 0.9662 - loss: 0.0996 - val_accuracy: 0.9631 - val_loss: 0.1119
Epoch 3/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 165s 707ms/step - accuracy: 0.9817 - loss: 0.0597 - val_accuracy: 0.9673 - val_loss: 0.1006
Epoch 4/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 172s 734ms/step - accuracy: 0.9861 - loss: 0.0470 - val_accuracy: 0.9684 - val_loss: 0.0986
Epoch 5/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 158s 677ms/step - accuracy: 0.9891 - loss: 0.0384 - val_accuracy: 0.9695 - val_loss: 0.0910
Epoch 6/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 155s 664ms/step - accuracy: 0.9920 - loss: 0.0290 - val_accuracy: 0.9732 - val_loss: 0.0870
Epoch 7/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 155s 662ms/step - accuracy: 0.9921 - loss: 0.0261 - val_accuracy: 0.9722 - val_loss: 0.0862
Epoch 8/10
234/234 ━━━━━━━━━━━━━━━━━━━━ 158s 675ms/step - accuracy: 0.9954 - lo

In [8]:
# Guardar modelo entrenado
model.save("../model/garbage_classifier.keras")
print("✅ Modelo guardado con éxito.")

✅ Modelo guardado con éxito.


In [26]:
# Parámetros para entrenamiento con food-101 (orgánicos)
DATASET_FOOD_DIR = "../datasets_food"  # Ajusta si es necesario
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

food_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator_food = food_datagen.flow_from_directory(
    DATASET_FOOD_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator_food = food_datagen.flow_from_directory(
    DATASET_FOOD_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)


Found 2448 images belonging to 1 classes.
Found 612 images belonging to 1 classes.


In [29]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# Modelo para residuos orgánicos (food-101)
base_model_food = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model_food.trainable = False

food_model = models.Sequential([
    base_model_food,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # <--- Solo una clase: orgánico
])

food_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
food_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [30]:
history_food = food_model.fit(
    train_generator_food,
    validation_data=val_generator_food,
    epochs=7,
    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)]
)


Epoch 1/7
77/77 ━━━━━━━━━━━━━━━━━━━━ 110s 1s/step - accuracy: 0.9425 - loss: 0.1216 - val_accuracy: 1.0000 - val_loss: 2.9202e-08
Epoch 2/7
77/77 ━━━━━━━━━━━━━━━━━━━━ 56s 717ms/step - accuracy: 1.0000 - loss: 1.7551e-06 - val_accuracy: 1.0000 - val_loss: 2.8707e-08
Epoch 3/7
77/77 ━━━━━━━━━━━━━━━━━━━━ 56s 722ms/step - accuracy: 1.0000 - loss: 7.8420e-07 - val_accuracy: 1.0000 - val_loss: 2.8475e-08
Epoch 4/7
77/77 ━━━━━━━━━━━━━━━━━━━━ 56s 731ms/step - accuracy: 1.0000 - loss: 1.3297e-06 - val_accuracy: 1.0000 - val_loss: 2.8088e-08
Epoch 5/7
77/77 ━━━━━━━━━━━━━━━━━━━━ 57s 734ms/step - accuracy: 1.0000 - loss: 1.9746e-06 - val_accuracy: 1.0000 - val_loss: 2.7651e-08
Epoch 6/7
77/77 ━━━━━━━━━━━━━━━━━━━━ 57s 731ms/step - accuracy: 1.0000 - loss: 1.2128e-06 - val_accuracy: 1.0000 - val_loss: 2.7255e-08
Epoch 7/7
77/77 ━━━━━━━━━━━━━━━━━━━━ 57s 733ms/step - accuracy: 1.0000 - loss: 1.0651e-06 - val_accuracy: 1.0000 - val_loss: 2.6795e-08


In [31]:
food_model.save("../model/organic_classifier.keras")
print("✅ Modelo guardado con éxito.")


✅ Modelo guardado con éxito.
